# Notebook to convert the Baysor output (poly.json) and spots.csv to tsv so they can be used in the viewer

The three files that are generated (cellCoords_0.tsv, geneData_0.tsv and cellData_0.tsv) should be 
copied in the relevant folder under dashboard/data/tsv/

In [1]:
import pandas as pd
import numpy as np

## Create cellCoords_0.tsv

In [2]:
cellCoords = pd.read_json('poly.json')
cellCoords['cell_id'] = np.arange(cellCoords.shape[0])
cellCoords['label'] = cellCoords['cell_id'] + 1

In [3]:
cellCoords = cellCoords.rename({0: '_coords'}, axis=1)

In [4]:
cellCoords = cellCoords[['cell_id', 'label', '_coords']]

In [5]:
coords = []
for index, row in cellCoords.iterrows():
    c = np.array(row['_coords']).astype(np.int64)
    coords.append(c.tolist())

In [6]:
cellCoords['coords'] = coords
cellCoords = cellCoords[['cell_id', 'label', 'coords']]
cellCoords.to_csv('cellCoords_0.tsv', index=False, sep='\t')

In [7]:
cellCoords.head()

,cell_id,label,coords
0,0,1,"[[84, 150], [79, 150], [74, 150], [70, 150], [..."
1,1,2,"[[204, 159], [199, 159], [194, 159], [194, 155..."
2,2,3,"[[70, 270], [74, 270], [79, 270], [84, 270], [..."
3,3,4,"[[137, 255], [141, 255], [146, 255], [151, 255..."
4,4,5,"[[290, 83], [290, 88], [295, 88], [299, 88], [..."


## Create geneData_0.tsv

In [8]:
geneData = pd.read_csv('spots_all.csv')
geneData['x'] = geneData.x.astype(np.int64)
geneData['y'] = geneData.y.astype(np.int64)


In [9]:
geneData = geneData.rename({'gene': 'Gene'}, axis=1)
_, Expt = np.unique(geneData.Gene, return_inverse=True)
geneData['Expt'] = Expt
# geneData = geneData[['Gene','Expt', 'x', 'y']]
# geneData

In [10]:
background_cell_id = cellCoords.cell_id.max() + 1
background_cell_id

6620

In [11]:
# geneData

In [12]:
neighbour = int(background_cell_id) * np.ones(geneData.shape[0])
neighbour_array = [[int(x)] for x in neighbour]
neighbour_prob = [[1.0] for x in neighbour]

In [13]:
geneData['neighbour'] = neighbour.astype(np.int)
geneData['neighbour_array'] = neighbour_array
geneData['neighbour_prob'] = neighbour_prob

In [14]:
geneData.to_csv('geneData_0.tsv', index=False, sep='\t')

In [15]:
geneData.head()

,x,y,Gene,Expt,neighbour,neighbour_array,neighbour_prob
0,8473,2043,Aldoc,0,6620,[6620],[1.0]
1,9208,2035,Aldoc,0,6620,[6620],[1.0]
2,9035,2025,Aldoc,0,6620,[6620],[1.0]
3,7982,2020,Aldoc,0,6620,[6620],[1.0]
4,8767,2014,Aldoc,0,6620,[6620],[1.0]


## Create cellData_0.tsv

In [16]:
cellData = pd.DataFrame({'Cell_Num': cellCoords.cell_id})
cellData['X'] = 0
cellData['Y'] = 0
cellData['Genenames'] = [[] for x in cellData.X]
cellData['CellGeneCount'] = [[] for x in cellData.X]
cellData['ClassName'] = [['Generic'] for x in cellData.X]
cellData['Prob'] = [[1.0] for x in cellData.X]

In [17]:
cellData.to_csv('cellData_0.tsv', index=False, sep='\t')
cellData.head()

,Cell_Num,X,Y,Genenames,CellGeneCount,ClassName,Prob
0,0,0,0,[],[],[Generic],[1.0]
1,1,0,0,[],[],[Generic],[1.0]
2,2,0,0,[],[],[Generic],[1.0]
3,3,0,0,[],[],[Generic],[1.0]
4,4,0,0,[],[],[Generic],[1.0]
